#### Importing useful libraries

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras import initializers, regularizers
from tensorflow.keras import optimizers

from tensorflow.keras import initializers, regularizers, optimizers
import tensorflow as tf
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.callbacks import ModelCheckpoint

#### Defining the architecture

In [ ]:
input_shape = (60,1)

ini = keras.initializers.he_normal()

final_model = Sequential()
final_model.add(GaussianNoise(0.07,input_shape=input_shape))
final_model.add(Conv1D(filters=7,
                 kernel_size=15,
                 kernel_regularizer=keras.regularizers.l2(0.01),
                 kernel_initializer=ini))
final_model.add(tf.keras.layers.PReLU(shared_axes=[1]))
final_model.add(MaxPooling1D(6, padding='same'))
final_model.add(Conv1D(filters=6,
                 kernel_size=3,
                 kernel_regularizer=keras.regularizers.l2(0.00008),
                 kernel_initializer=ini))
final_model.add(tf.keras.layers.PReLU(shared_axes=[1]))
final_model.add(Flatten())
final_model.add(Dense(8, kernel_regularizer=keras.regularizers.l2(0.00008), kernel_initializer=ini))
final_model.add(Dropout(0.16))
final_model.add(tf.keras.layers.PReLU(shared_axes=[1]))
final_model.add(Dense(3,activation='softmax'))

opt = optimizers.Nadam()
final_model.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=opt, metrics=['accuracy'])

In [ ]:
print(final_model.summary())

#### Loading the weights

In [ ]:
final_model.load_weights('MaFalDDa_parameters_B.h5')

#### Loading the data

In [ ]:
fname = 'timeseries_competition.csv'
x = np.loadtxt(fname,delimiter=',',dtype=float)
N = len(x)

# Subtract the mean of each series
xm_horiz = x.mean(axis=1)
for i in range(N):
    x[i] -= xm_horiz[i]
    
# Subtract the mean of each feature in order to have unbiased series
xm_vert = np.loadtxt('mean_features.txt')
for i in range(len(x[0])):
    x[:,i] -= xm_vert[i]
    
x_std = np.std(x)
x /= x_std

x = x.reshape(x.shape[0],60,1)

#### Generating the outcome

In [ ]:
y = final_model.predict_classes(x)
np.savetxt('MaFalDDa_predictions_B.ipynb', y, fmt='%i')
np.savetxt('MaFalDDa_predictions_B.txt', y, fmt='%i')